In [1]:
import yfinance as yf
import pandas_datareader

In [2]:
import sys
!{sys.executable} -m pip install yfinance


  Using cached yfinance-0.2.25-py2.py3-none-any.whl (63 kB)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
imageio 2.23.0 requires pillow>=8.3.2, but you have pillow 8.2.0 which is incompatible.


  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2021.1
    Uninstalling pytz-2021.1:
      Successfully uninstalled pytz-2021.1
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.1
    Uninstalling numpy-1.20.1:
      Successfully uninstalled numpy-1.20.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4
  Attempting uninstall: lxml
    Found existing instal


scipy 1.6.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.4 which is incompatible.


In [2]:
import yfinance as yf
import pandas as pd


In [12]:
!{sys.executable} -m pip install pandas_ta

  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218925 sha256=93a51a8cc7b7c775e93d61a4c96d01c11385df72375293c0e7d50490603e1074
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\54\4a\75\06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta


<h1> VERSION 1

In [1]:
import pandas_ta as ta
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime, timedelta

In [2]:
def trigger(df):
    i =0
    cross=[0]
    for i in range(len(df.index)):
        if(i!=0):
            #difference=df['macd_h'][i]-df['macd_h'][i-1]
            #cross.append(difference)
            cross.append(df["macd_h"][i])
        #else:
            #df["cross1"][0]=0
    df["cross"]=cross  
    alert=[]
    
    # assign the array alert to either golden , death or -
    
    for j in range(len(df.index)):
        if(j!=0):
            if(df["cross"][j]>=0 and df["cross"][j-1]<0):
                alert.append("golden")
            elif(df["cross"][j]<0 and df["cross"][j-1]>=0):
                alert.append("death")
            else:
                alert.append("-")
        else:
            alert.append("-")
            
    df["alert"]=alert    

In [3]:
def get_macd(name):
    # Request historic pricing data via finance.yahoo.com API
    #df = yf.Ticker(name).history(period='3mo')[['Close', 'Open', 'High', 'Volume', 'Low']]
    
    #df = df.dropna(subset=['Close']).drop(columns=['Dividends'])
    #df = pdr.get_data_yahoo(name, period='3mo')[['Close', 'Open', 'High', 'Volume', 'Low']]
    
    # Calculate the date range for the last 3 months
    end_date = datetime.today()
    start_date = end_date - timedelta(days=90)
    df = yf.download(name, start=start_date, end=end_date)[['Close', 'Open', 'High', 'Volume', 'Low']]
    # Request historic pricing data via finance.yahoo.com API
    #df = pdr.get_data_yahoo(name, start=start_date, end=end_date)[['Close', 'Open', 'High', 'Volume', 'Low']]
    
    if 'Dividends' in df.columns:
        df = df.drop(columns=['Dividends'])
    
    # # Calculate MACD values using the pandas_ta library
    # df.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)
    # Get the 26-day EMA of the closing price
    k = df['Close'].ewm(span=12, adjust=False, min_periods=12).mean()

    # Get the 12-day EMA of the closing price
    d = df['Close'].ewm(span=26, adjust=False, min_periods=26).mean()

    # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
    macd = k - d

    # Get the 9-Day EMA of the MACD for the Trigger line
    macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()

    # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
    macd_h = macd - macd_s

    # Add all of our new values for the MACD to the dataframe
    df['macd'] = df.index.map(macd)
    df['macd_h'] = df.index.map(macd_h)
    df['macd_s'] = df.index.map(macd_s)

    # View our data
    pd.set_option("display.max_columns", None)
    
    return df
    
    #print(df)


In [4]:
def display(df):
    trigger(df)
    display=df.loc[:,("Close","cross","alert")]
    print(display.tail())

In [9]:
get_macd("CAT")

[*********************100%***********************]  1 of 1 completed


,Close,Open,High,Volume,Low,macd,macd_h,macd_s
Date,,,,,,,,
2023-06-12,237.899994,235.229996,238.889999,3644200,233.320007,NaN,NaN,NaN
2023-06-13,245.440002,240.080002,246.679993,4335100,239.679993,NaN,NaN,NaN
2023-06-14,243.320007,246.800003,248.320007,3122400,241.570007,NaN,NaN,NaN
2023-06-15,247.669998,244.000000,248.380005,2743900,243.479996,NaN,NaN,NaN
2023-06-16,245.270004,249.399994,250.889999,5260800,244.970001,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2023-08-31,281.130005,283.640015,284.809998,2579000,280.839996,3.274402,-0.412260,3.686662
2023-09-01,286.250000,284.799988,287.070007,2335400,283.649994,3.778298,0.073309,3.704989
2023-09-05,281.630005,286.339996,289.410004,2515600,281.380005,3.761484,0.045196,3.716288


<h2> S&P 500

In [5]:
compName=pd.read_csv("constituents_csv.csv")
coName=compName["Symbol"]
coType=compName["Sector"]
#top20=coName[:100]
#top
#top20_list=top20.tolist()

coName_list=coName.tolist()
coType_list=coType.tolist()
print(coName_list)

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'AAP', 'AMD', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', '

# Incomplete (US)


In [21]:
i=0
for i in range(len(coName.index)):
    print("\nSymbol= "+coName_list[i])
    print("\nSector= "+coType_list[i])
    df=get_macd(coName_list[i])
    display(df)


Symbol= MMM

Sector= Industrials
[*********************100%***********************]  1 of 1 completed
                Close     cross alert
Date                                 
2023-11-13  92.690002  0.579496     -
2023-11-14  94.970001  0.654050     -
2023-11-15  96.820000  0.786815     -
2023-11-16  95.709999  0.758412     -
2023-11-17  95.339996  0.675977     -

Symbol= AOS

Sector= Industrials
[*********************100%***********************]  1 of 1 completed
                Close     cross alert
Date                                 
2023-11-13  73.379997  0.376410     -
2023-11-14  76.019997  0.548792     -
2023-11-15  75.000000  0.554750     -
2023-11-16  75.639999  0.560757     -
2023-11-17  76.379997  0.571583     -

Symbol= ABT

Sector= Health Care
[*********************100%***********************]  1 of 1 completed
                 Close     cross alert
Date                                  
2023-11-13   95.790001  0.240228     -
2023-11-14   97.080002  0.368945     -
202


1 Failed download:
['ABMD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-08-21 16:16:35.412786 -> 2023-11-19 16:16:35.412786)')



      Close  cross alert
Date                    
0       NaN      0     -

Symbol= ACN

Sector= Information Technology


<ipython-input-14-d1d63e55dac2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cross"]=cross
<ipython-input-14-d1d63e55dac2>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["alert"]=alert


[*********************100%***********************]  1 of 1 completed
                 Close     cross alert
Date                                  
2023-11-13  315.630005  2.465861     -
2023-11-14  320.489990  2.427930     -
2023-11-15  325.500000  2.588920     -
2023-11-16  327.320007  2.656673     -
2023-11-17  327.829987  2.572775     -

Symbol= ATVI

Sector= Communication Services
[*********************100%***********************]  1 of 1 completed
                Close     cross alert
Date                                 
2023-10-16  94.419998 -0.015723     -
2023-10-17  94.419998 -0.028052     -
2023-10-18  94.419998 -0.039805     -
2023-10-19  94.419998 -0.050489     -
2023-10-20  94.419998 -0.059829     -

Symbol= ADM

Sector= Consumer Staples
[*********************100%***********************]  1 of 1 completed
                Close     cross alert
Date                                 
2023-11-13  73.040001  0.328990     -
2023-11-14  73.860001  0.396726     -
2023-11-15  74.23

...

[*********************100%***********************]  1 of 1 completed
                Close     cross   alert
Date                                   
2023-11-13  58.270000 -0.191900       -
2023-11-14  60.299999 -0.103516       -
2023-11-15  60.090000 -0.059767       -
2023-11-16  60.700001  0.006178  golden
2023-11-17  60.560001  0.034700       -

Symbol= XLNX

Sector= Information Technology
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['XLNX']: Exception('%ticker%: No timezone found, symbol may be delisted')



      Close  cross alert
Date                    
0       NaN      0     -

Symbol= XYL

Sector= Industrials
[*********************100%***********************]  1 of 1 completed
                 Close     cross alert
Date                                  
2023-11-13   98.510002  0.844906     -
2023-11-14  101.199997  0.941778     -
2023-11-15  100.239998  0.883416     -
2023-11-16  101.260002  0.855163     -
2023-11-17  101.160004  0.773836     -

Symbol= YUM

Sector= Consumer Discretionary


<ipython-input-14-d1d63e55dac2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cross"]=cross
<ipython-input-14-d1d63e55dac2>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["alert"]=alert


[*********************100%***********************]  1 of 1 completed
                 Close     cross alert
Date                                  
2023-11-13  125.309998  0.653475     -
2023-11-14  127.529999  0.684472     -
2023-11-15  126.620003  0.605456     -
2023-11-16  127.830002  0.595540     -
2023-11-17  127.660004  0.540030     -

Symbol= ZBRA

Sector= Information Technology
[*********************100%***********************]  1 of 1 completed
                 Close     cross alert
Date                                  
2023-11-13  202.610001  1.156479     -
2023-11-14  215.660004  1.842974     -
2023-11-15  216.960007  2.314224     -
2023-11-16  215.479996  2.443171     -
2023-11-17  218.020004  2.601788     -

Symbol= ZBH

Sector= Health Care
[*********************100%***********************]  1 of 1 completed
                 Close     cross alert
Date                                  
2023-11-13  105.820000  0.397631     -
2023-11-14  108.070000  0.527614     -
2023-11-15 

<h2> KLSE 

In [6]:
#compNameKL=pd.read_csv("List of Stocks For An Exchange.csv")
#compNameKL=compNameKL["Local Symbol"]
#coNameKL_list=compNameKL.tolist()
coNameKL_list=["0012.KL", "0001.KL", "0002.KL", "0003.KL", "0005.KL", "0006.KL", "0007.KL", "0008.KL", "0009.KL", "0010.KL", "0011.KL", "0016.KL", "0017.KL", "0018.KL", "0020.KL", "0021.KL", "0022.KL", "0023.KL", "0024.KL", "0025.KL", "0026.KL", "0028.KL", "0029.KL", "0030.KL", "0032.KL", "0034.KL", "0035.KL", "0036.KL", "0037.KL", "0038.KL", "0039.KL", "0040.KL", "0041.KL", "0043.KL", "0045.KL", "0047.KL", "0048.KL", "0049.KL", "0050.KL", "0051.KL", "0053.KL", "0054.KL", "0055.KL", "0056.KL", "0058.KL", "0059.KL", "0060.KL", "0062.KL", "0064.KL", "0065.KL", "0066.KL", "0068.KL", "0069.KL", "0070.KL", "0071.KL", "0072.KL", "0074.KL", "0075.KL", "0076.KL", "0078.KL", "0079.KL", "0080.KL", "0081.KL", "0082.KL", "0083.KL", "0084.KL", "0085.KL", "0086.KL", "0088.KL", "0089.KL", "0090.KL", "0091.KL", "0092.KL", "0093.KL", "0094.KL", "0095.KL", "0096.KL", "0097.KL", "0098.KL", "0099.KL", "0100.KL", "0101.KL", "0102.KL", "0103.KL", "0104.KL", "0105.KL", "0106.KL", "0107.KL", "0108.KL", "0109.KL", "0110.KL", "0111.KL", "0112.KL", "0113.KL", "0116.KL", "0117.KL", "0118.KL", "0119.KL", "0120.KL", "0122.KL", "0123.KL", "0125.KL", "0126.KL", "0127.KL", "0128.KL", "0129.KL", "0130.KL", "0131.KL", "0132.KL", "0133.KL", "0135.KL", "0136.KL", "0137.KL", "0138.KL", "0140.KL", "0141.KL", "0143.KL", "0145.KL", "0146.KL", "0147.KL", "0148.KL", "0149.KL", "0150.KL", "0151.KL", "0152.KL", "0153.KL", "0154.KL", "0155.KL", "0156.KL", "0157.KL", "0158.KL", "0159.KL", "0162.KL", "0163.KL", "0165.KL", "0166.KL", "0167.KL", "0168.KL", "0169.KL", "0171.KL", "0173.KL", "0174.KL", "1007.KL", "1015.KL", "1023.KL", "1058.KL", "1066.KL", "1082.KL", "1147.KL", "1155.KL", "1163.KL", "1171.KL", "1198.KL", "1201.KL", "1236.KL", "1287.KL", "1295.KL", "1368.KL", "1473.KL", "1481.KL", "1503.KL", "1538.KL", "1562.KL", "1589.KL", "1597.KL", "1619.KL", "1643.KL", "1651.KL", "1694.KL", "1716.KL", "1724.KL",
                   "1783.KL", "1813.KL", "1818.KL", "1899.KL", "1902.KL", "1929.KL", "1945.KL", "1961.KL", "1996.KL", "2003.KL", "2011.KL", "2038.KL", "2054.KL", "2062.KL", "2089.KL", "2097.KL", "2127.KL", "2135.KL", "2143.KL", "2186.KL", "2194.KL", "2208.KL", "2216.KL", "2224.KL", "2259.KL", "2283.KL", "2291.KL", "2305.KL", "2313.KL", "2372.KL", "2429.KL", "2445.KL", "2453.KL", "2488.KL", "2542.KL", "2569.KL", "2577.KL", "2593.KL", "2607.KL", "2615.KL", "2658.KL", "2674.KL", "2682.KL", "2739.KL", "2747.KL", "2755.KL", "2771.KL", "2828.KL", "2836.KL", "2852.KL", "2879.KL", "2887.KL", "2925.KL", "2968.KL", "2976.KL", "2984.KL", "3018.KL", "3026.KL", "3034.KL", "3042.KL", "3069.KL", "3107.KL", "3115.KL", "3158.KL", "3174.KL", "3182.KL", "3204.KL", "3239.KL", "3247.KL", "3255.KL", "3298.KL", "3301.KL", "3336.KL", "3379.KL", "3395.KL", "3417.KL", "3441.KL", "3476.KL", "3484.KL", "3492.KL", "3514.KL", "3522.KL", "3557.KL", "3565.KL", "3573.KL", "3581.KL", "3611.KL", "3662.KL", "3689.KL", "3719.KL", "3743.KL", "3778.KL", "3786.KL", "3794.KL", "3816.KL", "3859.KL", "3867.KL", "3883.KL", "3891.KL", "3905.KL", "3913.KL", "3921.KL", "3948.KL", "4006.KL", "4022.KL", "4057.KL", "4065.KL", "4081.KL", "4103.KL", "4162.KL", "4197.KL", "4219.KL", "4235.KL", "4243.KL", "4251.KL", "4286.KL", "4316.KL", "4324.KL", "4359.KL", "4375.KL", "4383.KL", "4405.KL", "4421.KL", "4448.KL", "4456.KL", "4464.KL", "4502.KL", "4529.KL", "4537.KL", "4561.KL", "4588.KL", "4596.KL", "4634.KL", "4642.KL", "4677.KL", "4707.KL",
                   "4715.KL", "4723.KL", "4731.KL", "4758.KL", "4782.KL", "4804.KL", "4839.KL", "4847.KL", "4863.KL", "4871.KL", "4898.KL", "4936.KL", "4944.KL", "4952.KL", "4995.KL", "5000.KL", "5001.KL", "5005.KL", "5006.KL", "5007.KL", "5008.KL", "5009.KL", "5010.KL", "5011.KL", "5012.KL", "5014.KL", "5015.KL", "5016.KL", "5018.KL", "5019.KL", "5020.KL", "5021.KL", "5022.KL", "5024.KL", "5025.KL", "5026.KL", "5027.KL", "5028.KL", "5029.KL", "5031.KL", "5032.KL", "5035.KL", "5036.KL", "5037.KL", "5038.KL", "5040.KL", "5041.KL", "5042.KL", "5043.KL", "5047.KL", "5048.KL", "5049.KL", "5053.KL", "5054.KL", "5055.KL", "5056.KL", "5058.KL", "5059.KL", "5060.KL", "5062.KL", "5065.KL", "5066.KL", "5068.KL", "5069.KL", "5070.KL", "5071.KL", "5072.KL", "5073.KL", "5075.KL", "5077.KL", "5078.KL", "5079.KL", "5080.KL", "5081.KL", "5082.KL", "5084.KL", "5085.KL", "5087.KL", "5088.KL", "5089.KL", "5090.KL", "5091.KL", "5092.KL", "5094.KL", "5095.KL", "5096.KL", "5097.KL", "5098.KL", "5099.KL", "5100.KL", "5101.KL", "5102.KL", "5104.KL", "5105.KL", "5106.KL", "5107.KL", "5108.KL", "5109.KL", "5110.KL", "5111.KL", "5112.KL", "5113.KL", "5115.KL", "5116.KL", "5120.KL", "5121.KL", "5122.KL", "5123.KL", "5124.KL", "5125.KL", "5126.KL", "5127.KL", "5128.KL", "5129.KL", "5130.KL", "5131.KL", "5132.KL", "5133.KL", "5134.KL", "5135.KL", "5136.KL", "5138.KL", "5139.KL", "5140.KL", "5141.KL", "5142.KL", "5143.KL", "5145.KL", "5146.KL", "5147.KL", "5148.KL", "5149.KL", "5150.KL", "5151.KL", "5152.KL", "5155.KL", "5156.KL", "5157.KL", "5158.KL", "5159.KL", "5160.KL", "5161.KL", "5162.KL", "5163.KL", "5165.KL", "5166.KL", "5167.KL", "5168.KL", "5169.KL", "5170.KL", "5171.KL", "5172.KL", "5173.KL", "5175.KL", "5176.KL", "5178.KL", "5180.KL", "5181.KL", "5182.KL", "5183.KL", "5184.KL", "5185.KL", "5186.KL", "5187.KL", "5188.KL", "5189.KL", "5190.KL", "5191.KL", "5192.KL", "5194.KL", "5195.KL", "5196.KL", "5197.KL", "5199.KL", "5200.KL", "5201.KL", "5202.KL", "5204.KL", "5205.KL", "5207.KL", "5210.KL", "5211.KL", "5212.KL",
                   "5213.KL", "5214.KL", "5215.KL", "5223.KL", "5231.KL", "5258.KL", "5274.KL", "5304.KL", "5339.KL", "5347.KL", "5355.KL", "5371.KL", "5398.KL", "5401.KL", "5436.KL", "5509.KL", "5517.KL", "5533.KL", "5568.KL", "5576.KL", "5584.KL", "5592.KL", "5606.KL", "5614.KL", "5622.KL", "5649.KL", "5657.KL", "5665.KL", "5673.KL", "5681.KL", "5703.KL", "5711.KL", "5738.KL", "5754.KL", "5789.KL", "5797.KL", "5819.KL", "5827.KL", "5835.KL", "5843.KL", "5878.KL", "5886.KL", "5908.KL", "5916.KL", "5924.KL", "5932.KL", "5959.KL", "5975.KL", "5983.KL", "6009.KL", "6012.KL", "6017.KL", "6025.KL", "6033.KL", "6041.KL", "6068.KL", "6076.KL", "6084.KL", "6114.KL", "6139.KL", "6149.KL", "6157.KL", "6173.KL", "6181.KL", "6203.KL", "6211.KL", "6238.KL", "6246.KL", "6254.KL", "6262.KL", "6297.KL", "6327.KL", "6351.KL", "6378.KL", "6432.KL", "6459.KL", "6483.KL", "6491.KL", "6505.KL", "6521.KL", "6548.KL", "6556.KL", "6572.KL", "6599.KL", "6602.KL", "6637.KL", "6645.KL", "6653.KL", "6661.KL", "6688.KL", "6718.KL", "6742.KL", "6769.KL", "6807.KL", "6815.KL", "6866.KL", "6874.KL", "6888.KL", "6904.KL", "6912.KL", "6939.KL", "6947.KL", "6963.KL", "6971.KL", "6998.KL", "7002.KL", "7003.KL", "7004.KL", "7005.KL", "7006.KL", "7007.KL", "7008.KL", "7010.KL", "7013.KL", "7014.KL", "7016.KL", "7017.KL", "7018.KL", "7020.KL", "7022.KL", "7023.KL", "7025.KL", "7027.KL", "7028.KL", "7029.KL", "7031.KL", "7033.KL", "7034.KL", "7035.KL", "7036.KL", "7042.KL", "7043.KL", "7044.KL", "7045.KL", "7047.KL", "7048.KL", "7050.KL", "7051.KL", "7052.KL", "7053.KL", "7054.KL", "7055.KL", "7059.KL", "7060.KL", "7061.KL", "7062.KL", "7064.KL", "7066.KL", "7070.KL", "7071.KL", "7073.KL", "7075.KL", "7076.KL", "7077.KL", "7078.KL", "7079.KL", "7080.KL", "7081.KL", "7082.KL", "7083.KL", "7084.KL", "7085.KL", "7086.KL", "7087.KL", "7088.KL", "7089.KL", "7090.KL", "7091.KL", "7094.KL", "7095.KL", "7096.KL", "7097.KL", "7098.KL", "7099.KL", "7100.KL", "7103.KL", "7104.KL", "7105.KL", "7106.KL", "7107.KL", "7108.KL", "7109.KL", "7110.KL", "7111.KL", "7112.KL", "7113.KL", "7114.KL", "7115.KL", "7117.KL",
                   "7118.KL", "7119.KL", "7120.KL", "7121.KL", "7122.KL", "7123.KL", "7125.KL", "7126.KL", "7128.KL", "7129.KL", "7130.KL", "7131.KL", "7132.KL", "7133.KL", "7134.KL", "7136.KL", "7137.KL", "7139.KL", "7140.KL", "7141.KL", "7143.KL", "7145.KL", "7146.KL", "7148.KL", "7149.KL", "7152.KL", "7153.KL", "7154.KL", "7155.KL", "7157.KL", "7158.KL", "7160.KL", "7161.KL", "7162.KL", "7163.KL", "7164.KL", "7165.KL", "7167.KL", "7168.KL", "7169.KL", "7170.KL", "7171.KL", "7172.KL", "7173.KL", "7174.KL", "7176.KL", "7178.KL", "7179.KL", "7180.KL", "7181.KL", "7182.KL", "7183.KL", "7184.KL", "7185.KL", "7186.KL", "7187.KL", "7188.KL", "7189.KL", "7190.KL", "7191.KL", "7192.KL", "7193.KL", "7195.KL", "7196.KL", "7197.KL", "7198.KL", "7199.KL", "7200.KL", "7201.KL", "7202.KL", "7203.KL", "7204.KL", "7205.KL", "7206.KL", "7207.KL", "7208.KL", "7209.KL", "7210.KL", "7211.KL", "7212.KL", "7213.KL", "7214.KL", "7215.KL", "7216.KL", "7217.KL", "7218.KL", "7219.KL", "7220.KL", "7221.KL", "7222.KL", "7223.KL", "7225.KL", "7226.KL", "7227.KL", "7228.KL", "7229.KL", "7230.KL", "7231.KL", "7232.KL", "7233.KL", "7234.KL", "7235.KL", "7236.KL", "7237.KL", "7239.KL", "7240.KL", "7241.KL", "7242.KL", "7243.KL", "7245.KL", "7246.KL", "7247.KL", "7248.KL", "7249.KL", "7250.KL", "7251.KL", "7252.KL", "7253.KL", "7277.KL", "7285.KL", "7293.KL", "7315.KL", "7323.KL", "7358.KL", "7366.KL", "7374.KL", "7382.KL", "7412.KL", "7439.KL", "7455.KL", "7471.KL", "7498.KL", "7501.KL", "7528.KL", "7544.KL", "7579.KL", "7595.KL", "7609.KL", "7617.KL", "7641.KL", "7668.KL", "7676.KL", "7692.KL", "7722.KL", "7757.KL", "7765.KL", "7773.KL", "7781.KL", "7803.KL", "7811.KL", "7854.KL", "7889.KL", "7927.KL", "7935.KL", "7943.KL", "7986.KL", "8044.KL", "8052.KL", "8079.KL",
                   "8087.KL", "8095.KL", "8117.KL", "8125.KL", "8133.KL", "8141.KL", "8176.KL", "8192.KL", "8206.KL", "8273.KL", "8281.KL", "8303.KL", "8311.KL", "8338.KL", "8346.KL", "8362.KL", "8397.KL", "8419.KL", "8435.KL", "8443.KL", "8478.KL", "8486.KL", "8494.KL", "8508.KL", "8516.KL", "8524.KL", "8532.KL", "8567.KL", "8575.KL", "8583.KL", "8591.KL", "8605.KL", "8613.KL", "8621.KL", "8648.KL", "8656.KL", "8664.KL", "8672.KL", "8702.KL", "8737.KL", "8745.KL", "8761.KL", "8818.KL", "8826.KL", "8834.KL", "8842.KL", "8869.KL", "8877.KL", "8885.KL", "8893.KL", "8907.KL", "8923.KL", "8931.KL", "8966.KL", "8982.KL", "9008.KL", "9016.KL", "9059.KL", "9075.KL", "9083.KL", "9091.KL", "9113.KL", "9121.KL", "9148.KL", "9172.KL", "9199.KL", "9202.KL", "9237.KL", "9261.KL", "9288.KL", "9296.KL", "9318.KL", "9326.KL", "9334.KL", "9342.KL", "9369.KL", "9377.KL", "9385.KL", "9393.KL", "9407.KL", "9415.KL", "9423.KL", "9431.KL", "9458.KL", "9466.KL", "9474.KL", "9504.KL", "9539.KL", "9547.KL", "9555.KL", "9571.KL", "9598.KL", "9601.KL", "9628.KL", "9652.KL", "9679.KL", "9687.KL", "9695.KL", "9717.KL", "9725.KL", "9741.KL", "9776.KL", "9792.KL", "9806.KL", "9814.KL", "9822.KL", "9849.KL", "9865.KL", "9873.KL", "9881.KL", "9911.KL", "9938.KL"]
#coNameKL_list=str(coNameKL_list)
i =0
type(coNameKL_list[0])

        

str

# Incomplete KL

In [7]:
type(coNameKL_list[0])
coNameKL_list[0]
get_macd(coNameKL_list[0])
for i in range(len(coNameKL_list)):
    print("\nSymbol= "+coNameKL_list[i])
    if(coNameKL_list[i].isnumeric()):
        name=coNameKL_list[i]+".KL"
        get_macd(name)
    else:
        get_macd(coNameKL_list[i])

[*********************100%***********************]  1 of 1 completed

Symbol= 0012.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0001.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0002.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0003.KL
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['0003.KL']: Exception('%ticker%: No timezone found, symbol may be delisted')




Symbol= 0005.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0006.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0007.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0008.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0009.KL
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['0009.KL']: Exception('%ticker%: No timezone found, symbol may be delisted')




Symbol= 0010.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0011.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0016.KL
[*********************100%***********************]  1 of 1 completed

...



Symbol= 9865.KL

[*********************100%***********************]  1 of 1 completed


1 Failed download:
['9865.KL']: Exception('%ticker%: No timezone found, symbol may be delisted')




Symbol= 9873.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 9881.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 9911.KL
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['9911.KL']: Exception('%ticker%: No timezone found, symbol may be delisted')




Symbol= 9938.KL
[*********************100%***********************]  1 of 1 completed


# Only show golden cross (US)

In [6]:
for i in range(len(coName.index)):
    print("\nSymbol= "+coName_list[i])
    df=get_macd(coName_list[i])
    trigger(df)
    table=df.loc[:,("Close","cross","alert")]
    alert=df["alert"].values.tolist()
    alert=alert[-5:]
    if("golden" in alert):
        display(df) 


Symbol= MMM
[*********************100%***********************]  1 of 1 completed

Symbol= AOS
[*********************100%***********************]  1 of 1 completed

Symbol= ABT
[*********************100%***********************]  1 of 1 completed

Symbol= ABBV
[*********************100%***********************]  1 of 1 completed

Symbol= ABMD
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['ABMD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-09-04 00:30:54.013331 -> 2023-12-03 00:30:54.013331)')




Symbol= ACN


<ipython-input-2-d1d63e55dac2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cross"]=cross
<ipython-input-2-d1d63e55dac2>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["alert"]=alert


[*********************100%***********************]  1 of 1 completed

Symbol= ATVI
[*********************100%***********************]  1 of 1 completed

Symbol= ADM
[*********************100%***********************]  1 of 1 completed

Symbol= ADBE
[*********************100%***********************]  1 of 1 completed

Symbol= AAP
[*********************100%***********************]  1 of 1 completed

Symbol= AMD
[*********************100%***********************]  1 of 1 completed

Symbol= AES
[*********************100%***********************]  1 of 1 completed

Symbol= AFL
[*********************100%***********************]  1 of 1 completed

Symbol= A
[*********************100%***********************]  1 of 1 completed

Symbol= APD
[*********************100%***********************]  1 of 1 completed

Symbol= AKAM
[*********************100%***********************]  1 of 1 completed

Symbol= ALK
[*********************100%***********************]  1 of 1 completed

Symbol= ALB
[**************

...

[*********************100%***********************]  1 of 1 completed

Symbol= XEL
[*********************100%***********************]  1 of 1 completed
                Close     cross   alert
Date                                   
2023-11-27  60.450001  0.001154       -
2023-11-28  60.970001  0.028427       -
2023-11-29  60.000000 -0.021966   death
2023-11-30  60.840000 -0.002836       -
2023-12-01  61.430000  0.042918  golden

Symbol= XLNX
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['XLNX']: Exception('%ticker%: No timezone found, symbol may be delisted')




Symbol= XYL
[*********************100%***********************]  1 of 1 completed

<ipython-input-2-d1d63e55dac2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cross"]=cross
<ipython-input-2-d1d63e55dac2>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["alert"]=alert




Symbol= YUM
[*********************100%***********************]  1 of 1 completed

Symbol= ZBRA
[*********************100%***********************]  1 of 1 completed

Symbol= ZBH
[*********************100%***********************]  1 of 1 completed

Symbol= ZION
[*********************100%***********************]  1 of 1 completed

Symbol= ZTS
[*********************100%***********************]  1 of 1 completed


# Only show golden cross (KL)



In [7]:
for i in range(len(coNameKL_list)):
    print("\nSymbol= "+coNameKL_list[i])
    df=get_macd(coNameKL_list[i])
    trigger(df)
    table=df.loc[:,("Close","cross","alert")]
    alert=df["alert"].values.tolist()
    alert=alert[-5:]
    if("golden" in alert):
        display(df) 
       


Symbol= 0012.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0001.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0002.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0003.KL
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['0003.KL']: Exception('%ticker%: No timezone found, symbol may be delisted')




Symbol= 0005.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0006.KL


<ipython-input-2-d1d63e55dac2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cross"]=cross
<ipython-input-2-d1d63e55dac2>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["alert"]=alert


[*********************100%***********************]  1 of 1 completed
            Close     cross   alert
Date                               
2024-02-19  0.795 -0.003792       -
2024-02-20  0.820 -0.001699       -
2024-02-21  0.820 -0.000273       -
2024-02-22  0.820  0.000659  golden
2024-02-23  0.795 -0.000362   death

Symbol= 0007.KL
[*********************100%***********************]  1 of 1 completed
            Close     cross   alert
Date                               
2024-02-20  0.040  0.000275       -
2024-02-21  0.040  0.000074       -
2024-02-22  0.040 -0.000054   death
2024-02-23  0.050  0.000508  golden
2024-02-26  0.045  0.000515       -

Symbol= 0008.KL
[*********************100%***********************]  1 of 1 completed

Symbol= 0009.KL
[*********************100%***********************]  1 of 1 completed

...

[*********************100%***********************]  1 of 1 completed

Symbol= 9881.KL
[*********************100%***********************]  1 of 1 completed
            Close     cross   alert
Date                               
2024-02-20  0.465 -0.001270       -
2024-02-21  0.470 -0.000437       -
2024-02-22  0.460 -0.000516       -
2024-02-23  0.460 -0.000525       -
2024-02-26  0.480  0.000789  golden

Symbol= 9911.KL
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['9911.KL']: Exception('%ticker%: No timezone found, symbol may be delisted')




Symbol= 9938.KL

[*********************100%***********************]  1 of 1 completed

<ipython-input-2-d1d63e55dac2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cross"]=cross
<ipython-input-2-d1d63e55dac2>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["alert"]=alert



            Close     cross   alert
Date                               
2024-02-20  0.215  0.000287       -
2024-02-21  0.210 -0.000086   death
2024-02-22  0.215  0.000009  golden
2024-02-23  0.215  0.000073       -
2024-02-26  0.215  0.000114       -


In [20]:
a=compNameKL[2]+".KL"
a

'5347.KL'

# Version 2  

In [2]:
import pandas_ta as ta
import yfinance as yf
import pandas as pd

In [32]:
def trigger(df):
    i =0
    cross=[0]
    for i in range(len(df.index)):
        if(i!=0):
            difference=df['MACDs_12_26_9'][i]-df['MACDs_12_26_9'][i-1]
            cross.append(difference)
            
        #else:
            #df["cross1"][0]=0
    df["cross"]=cross  
    alert=[]
    for j in range(len(df.index)):
        if(j!=0):
            if(df["cross"][j]>=0 and df["cross"][j-1]<0):
                alert.append("golden")
            elif(df["cross"][j]<0 and df["cross"][j-1]>=0):
                alert.append("death")
            else:
                alert.append("-")
        else:
            alert.append("-")
            
    df["alert"]=alert    
            

In [33]:
def get_macd2(name):
    # Request historic pricing data via finance.yahoo.com API
    df = yf.Ticker(name).history(period='3mo')[['Close', 'Open', 'High', 'Volume', 'Low']]
    # Calculate MACD values using the pandas_ta library
    df.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)
    # View result
    pd.set_option("display.max_columns", None)  # show all columns
    #print(df['MACDs_12_26_9'])
    
    #print(df['MACDs_12_26_9'][-1])
    
    #for i in range(len(df.index)):
        #df['new line']=df['MACDs_12_26_9'] +5
    trigger(df)    
    #print(df['cross'])
    
    #print("----------------alert----------------")
    
    #print(df["alert"])
    display=df.loc[:,("cross","alert")]
    #print(display.tail())
    print(df)

In [34]:
get_macd2("AAPL")

                                Close        Open        High    Volume  \
Date                                                                      
2023-04-24 00:00:00-04:00  165.101639  164.772093  165.371270  41949600   
2023-04-25 00:00:00-04:00  163.543793  164.961829  166.080278  48714100   
2023-04-26 00:00:00-04:00  163.533798  162.834768  165.051703  45498800   
2023-04-27 00:00:00-04:00  168.177383  164.961830  168.327170  64902300   
2023-04-28 00:00:00-04:00  169.445618  168.257274  169.615396  55209200   
...                               ...         ...         ...       ...   
2023-07-17 00:00:00-04:00  193.990005  191.899994  194.320007  50520200   
2023-07-18 00:00:00-04:00  193.729996  193.350006  194.330002  48353800   
2023-07-19 00:00:00-04:00  195.100006  193.100006  198.229996  80507300   
2023-07-20 00:00:00-04:00  193.130005  195.089996  196.470001  59581200   
2023-07-21 00:00:00-04:00  191.940002  194.100006  194.970001  71917800   

                        

In [6]:
import pandas as pd
compName=pd.read_csv("constituents_csv.csv")
coName=compName["Symbol"]
coType=compName["Sector"]
#top20=coName[:100]
#top
#top20_list=top20.tolist()

coName_list=coName.tolist()
coType_list=coType.tolist()
print(coName_list)

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'AAP', 'AMD', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', '

In [12]:
i=0
for i in range(len(coName.index)):
    print("\nSymbol= "+coName_list[i])
    print("\nSector= "+coType_list[i])
    get_macd2(coName_list[i])


Symbol= MMM

Sector= Industrials
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.090785     -
2023-07-18 00:00:00-04:00  0.105556     -
2023-07-19 00:00:00-04:00  0.116689     -
2023-07-20 00:00:00-04:00  0.130117     -
2023-07-21 00:00:00-04:00  0.131698     -

Symbol= AOS

Sector= Industrials
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.066716     -
2023-07-18 00:00:00-04:00  0.094555     -
2023-07-19 00:00:00-04:00  0.106703     -
2023-07-20 00:00:00-04:00  0.101595     -
2023-07-21 00:00:00-04:00  0.084272     -

Symbol= ABT

Sector= Health Care
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.024140     -
2023-07-18 00:00:00-04:00  0.013705     -
2023-07-19 00:00:00-04:00  0.010950     -
2023-07-20 00:00:00-04:00  0.080456     -
2023-07-21 00:00:00-04:00  0.158082     -

Symbol= ABBV

S

ABMD: No price data found, symbol may be delisted (period=3mo)
<ipython-input-3-aee199348721>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cross"]=cross
<ipython-input-3-aee199348721>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["alert"]=alert


      cross alert
Date             
0         0     -

Symbol= ACN

Sector= Information Technology
                              cross  alert
Date                                      
2023-07-17 00:00:00-04:00  0.119132      -
2023-07-18 00:00:00-04:00  0.205478      -
2023-07-19 00:00:00-04:00  0.181805      -
2023-07-20 00:00:00-04:00  0.046925      -
2023-07-21 00:00:00-04:00 -0.012529  death

Symbol= ATVI

Sector= Communication Services
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.213292     -
2023-07-18 00:00:00-04:00  0.217041     -
2023-07-19 00:00:00-04:00  0.194792     -
2023-07-20 00:00:00-04:00  0.167023     -
2023-07-21 00:00:00-04:00  0.129370     -

Symbol= ADM

Sector= Consumer Staples
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.088081     -
2023-07-18 00:00:00-04:00  0.095551     -
2023-07-19 00:00:00-04:00  0.096495     -
2023-07-20 

                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.100873     -
2023-07-18 00:00:00-04:00  0.061582     -
2023-07-19 00:00:00-04:00  0.045132     -
2023-07-20 00:00:00-04:00  0.055723     -
2023-07-21 00:00:00-04:00  0.077157     -

Symbol= AXP

Sector= Financials
                              cross   alert
Date                                       
2023-07-17 00:00:00-04:00 -0.036327       -
2023-07-18 00:00:00-04:00  0.014102  golden
2023-07-19 00:00:00-04:00  0.024052       -
2023-07-20 00:00:00-04:00  0.022573       -
2023-07-21 00:00:00-04:00 -0.095810   death

Symbol= AIG

Sector= Financials
                              cross   alert
Date                                       
2023-07-17 00:00:00-04:00  0.002878  golden
2023-07-18 00:00:00-04:00  0.009106       -
2023-07-19 00:00:00-04:00  0.006283       -
2023-07-20 00:00:00-04:00  0.019778       -
2023-07-21 00:00:00-04:00  0.020253       -

Symbol= AMT

Sector= Re

ANTM: No data found, symbol may be delisted


...

BRK.B: No data found, symbol may be delisted


      cross alert
Date             
0         0     -

Symbol= BBY

Sector= Consumer Discretionary
                              cross   alert
Date                                       
2023-07-17 00:00:00-04:00 -0.068819       -
2023-07-18 00:00:00-04:00 -0.034317       -
2023-07-19 00:00:00-04:00 -0.004448       -
2023-07-20 00:00:00-04:00  0.001018  golden
2023-07-21 00:00:00-04:00 -0.013248   death

Symbol= BIO

Sector= Health Care
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.737276     -
2023-07-18 00:00:00-04:00  0.497906     -
2023-07-19 00:00:00-04:00  0.517836     -
2023-07-20 00:00:00-04:00  0.517459     -
2023-07-21 00:00:00-04:00  0.739800     -

Symbol= TECH

Sector= Health Care
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.142913     -
2023-07-18 00:00:00-04:00  0.098401     -
2023-07-19 00:00:00-04:00  0.088969     -
2023-07-20 00:00:00-

BF.B: No price data found, symbol may be delisted (period=3mo)
<ipython-input-3-aee199348721>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cross"]=cross
<ipython-input-3-aee199348721>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["alert"]=alert


      cross alert
Date             
0         0     -

Symbol= CHRW

Sector= Industrials
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.070968     -
2023-07-18 00:00:00-04:00  0.057170     -
2023-07-19 00:00:00-04:00  0.083919     -
2023-07-20 00:00:00-04:00  0.099707     -
2023-07-21 00:00:00-04:00  0.104380     -

Symbol= CDNS

Sector= Information Technology
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.179151     -
2023-07-18 00:00:00-04:00  0.222313     -
2023-07-19 00:00:00-04:00  0.204176     -
2023-07-20 00:00:00-04:00  0.114600     -
2023-07-21 00:00:00-04:00  0.108648     -

Symbol= CZR

Sector= Consumer Discretionary
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.071142     -
2023-07-18 00:00:00-04:00  0.114495     -
2023-07-19 00:00:00-04:00  0.153534     -
2023-07-20 00:00:00-04

Exception: The following 'Dividends' events are out-of-range, did not expect with interval 1d: DatetimeIndex(['2023-04-21 00:00:00-04:00', '2023-07-19 00:00:00-04:00'], dtype='datetime64[ns, America/New_York]', freq=None)

# Index macd

In [8]:
s_and_p_index=["^SP500-45","^SP500-35","^SP500-40","^SP500-50","^SP500-10","^SP500-15","^SP500-20","^SP500-4010"]
name=["IT","Health care","Financial","TE","Energy","Material","Industrial","Banking"]
i=0
for i in range(len(s_and_p_index)):
    print("\nSymbol= "+s_and_p_index[i])
    print("\nSector= "+name[i])
    get_macd2(s_and_p_index[i])


Symbol= ^SP500-45

Sector= IT
                              cross   alert
Date                                       
2023-07-17 00:00:00-04:00  0.141297  golden
2023-07-18 00:00:00-04:00  1.208671       -
2023-07-19 00:00:00-04:00  1.538951       -
2023-07-20 00:00:00-04:00  0.484990       -
2023-07-21 00:00:00-04:00 -0.497452   death

Symbol= ^SP500-35

Sector= Health care
                              cross   alert
Date                                       
2023-07-17 00:00:00-04:00  0.002847  golden
2023-07-18 00:00:00-04:00  0.268322       -
2023-07-19 00:00:00-04:00  0.545181       -
2023-07-20 00:00:00-04:00  1.098686       -
2023-07-21 00:00:00-04:00  1.641384       -

Symbol= ^SP500-40

Sector= Financial
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.354208     -
2023-07-18 00:00:00-04:00  0.443275     -
2023-07-19 00:00:00-04:00  0.503609     -
2023-07-20 00:00:00-04:00  0.564916     -
2023-07-21 00:00:00-04:

^SP500-10: No data found, symbol may be delisted


      cross alert
Date             
0         0     -

Symbol= ^SP500-15

Sector= Material
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.213381     -
2023-07-18 00:00:00-04:00  0.248847     -
2023-07-19 00:00:00-04:00  0.202308     -
2023-07-20 00:00:00-04:00  0.170324     -
2023-07-21 00:00:00-04:00  0.128332     -

Symbol= ^SP500-20

Sector= Industrial
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.187438     -
2023-07-18 00:00:00-04:00  0.198100     -
2023-07-19 00:00:00-04:00  0.153081     -
2023-07-20 00:00:00-04:00  0.142818     -
2023-07-21 00:00:00-04:00  0.026281     -

Symbol= ^SP500-4010

Sector= Banking
                              cross alert
Date                                     
2023-07-17 00:00:00-04:00  0.320429     -
2023-07-18 00:00:00-04:00  0.426151     -
2023-07-19 00:00:00-04:00  0.548261     -
2023-07-20 00:00:00-04:00  0.62688

In [16]:
get_macd("CAT")

                                Close        Open        High   Volume  \
Date                                                                     
2022-07-22 00:00:00-04:00  174.771698  177.579875  178.861649  1961500   
2022-07-25 00:00:00-04:00  177.892960  175.309838  178.665948  1845600   
2022-07-26 00:00:00-04:00  177.325470  177.256986  179.693330  1767700   
2022-07-27 00:00:00-04:00  181.258850  178.979047  182.168806  1856500   
2022-07-28 00:00:00-04:00  183.793045  182.824384  185.436858  2229200   
...                               ...         ...         ...      ...   
2023-07-17 00:00:00-04:00  256.191284  253.017017  257.574449  2674500   
2023-07-18 00:00:00-04:00  262.509979  256.360441  262.858261  3833400   
2023-07-19 00:00:00-04:00  262.750000  260.140015  262.920013  2771900   
2023-07-20 00:00:00-04:00  261.089996  264.170013  265.399994  2102300   
2023-07-21 00:00:00-04:00  257.649994  261.750000  261.760010  2249400   

                                  Low